# Combining Everything
Let's put everything together! Generate a playlist, get the songs in it, and give it a name! 

Note: we should do this all in functions for Django

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import argparse
import logging
import spotipy
import pandas as pd
import numpy as np
import requests
from spotipy import oauth2
import re
import os
import openai
import credentials

In [2]:
# Gain access to mainupation
# This will probably be gained on the intro page
def get_logged_in():
    SCOPE = ('user-library-read,user-read-currently-playing,playlist-read-private,playlist-modify-private,playlist-modify-public,user-modify-playback-state,user-read-private,user-read-playback-state')
    sp_oauth = oauth2.SpotifyOAuth(credentials.SPOTIPY_CLIENT_ID,credentials.SPOTIPY_CLIENT_SECRET, credentials.SPOTIPY_REDIRECT_URI ,scope=SCOPE )

    #click "Accept" in your browser when the auth window pops up
    code = sp_oauth.get_auth_response(open_browser=True)
    token = sp_oauth.get_access_token(code)
    refresh_token = token['refresh_token']
    sp = spotipy.Spotify(auth=token['access_token'])
    return sp

In [3]:
def find_and_filter(settings, genres_list, sp):
    #FINDING
    results = sp.recommendations(seed_genres=genres_list, limit=100)
    song_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    song_df = pd.DataFrame(columns = song_features_list)
    tracks = results["tracks"]

    for track in tracks:
        # Create empty dict
        playlist_features = {}
        playlist_features["artist"] = track["album"]["artists"][0]["name"]
        playlist_features["album"] = track["album"]["name"]
        playlist_features["track_name"] = track["name"]
        playlist_features["track_id"] = track["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in song_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        song_df = pd.concat([song_df, track_df], ignore_index = True)
    
    #FILTERING
    for index, setting in settings.iterrows():
        if setting["On"]:
            if setting["Level"] > 0:
                song_df = song_df[(song_df[setting["Name"]] >= song_df[setting["Name"]].mean())]
            elif setting["Level"] < 0:
                song_df = song_df[(song_df[setting["Name"]] <= song_df[setting["Name"]].mean())]
            else: #medium
                var = song_df[setting["Name"]].var()
                song_df = song_df[(song_df[setting["Name"]] >= song_df[setting["Name"]].mean()-2*var) & (song_df[setting["Name"]] <= song_df[setting["Name"]].mean()+2*var)]
    return song_df

In [4]:
def gather_songs(sp, generes_list, settings_df, goal):
    final_df = find_and_filter(settings_df,generes_list,sp)
    while (len(final_df) < goal):
        getter = find_and_filter(settings_df,generes_list,sp)
        final_df = pd.concat([final_df, getter], ignore_index = True)
    return final_df


In [10]:
def generate_playlist(sp,generes_list,settings_df, goal):
    songs = gather_songs(sp,generes_list,settings_df, goal)
    input = "The playlist has the following songs:"
    i = 0
    while i < len(songs) and i < 20:
        song_name = songs['track_name'][i]
        song_artist = songs['artist'][i] # only the first artist
        input = input + "\n- " +song_name + " by " + song_artist
        i = i + 1
    openai.api_key = credentials.api_key
    output = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "You are a playlist reccomendation software. The user will ask for a playlist title given a list of songs in the playlist."},
                {"role": "user", "content": input},
                {"role": "assistant", "content": "A playlist title should not be longer than 7 words and at minimum 2 words"},
                {"role": "assistant", "content": "Give more than 3 suggestions and less than 8"},
                {"role": "assistant", "content": "Structure your format in the output. ' here are three suggestions for playlist titles based on the songs you provided:\n1.Reccomendation\n2.Reccomendation\n3.Reccomendation\n4.Reccomendation\n5.Reccomendation, and so on'"}
            ]
        ) 
    reccomendation = output['choices'][0]['message']['content']
    print(reccomendation)
    #print('reccc',reccomendation)
    bullet_points = reccomendation.split('\n\n')[0].split('\n')[0:]
    playlist_titles = [point[2:] for point in bullet_points]
    playlist_titles = playlist_titles[1:]
    print(playlist_titles)
    best_title = playlist_titles[0]

    username = sp.current_user()['id']
    result = sp.user_playlist_create(username, name=best_title, description="A personalized discovery playlist created with Creatify")
    playlist_id = result['id']

    logger = logging.getLogger('examples.add_tracks_to_playlist')
    logging.basicConfig(level='DEBUG')
    scope = 'playlist-modify-public'

    tracks = songs["track_id"]

    sp.user_playlist_add_tracks(username, playlist_id=playlist_id, tracks=tracks)

    return playlist_titles, playlist_id

In [11]:
# VARIABLES TO CHANGE
#settings tester
settings = [{'Name': 'danceability', 'On': True, 'Level': 71}, {'Name': 'energy', 'On': True, 'Level': 69}, {'Name': 'valence', 'On': True, 'Level': 65}, {'Name': 'instrumentalness', 'On': False, 'Level': 0}, {'Name': 'tempo', 'On': False, 'Level': 0}]

settings_df = pd.DataFrame(settings)

goal = 20
generes_list = ['indie', 'indie-Pop', 'pop']
sp = get_logged_in()
titles, i = generate_playlist(sp,generes_list,settings_df, goal)

/var/folders/pz/lc5f5cgd1m5_ghmjddlz8r600000gn/T/ipykernel_49001/904931709.py:9: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = sp_oauth.get_access_token(code)


Based on the songs in your playlist, here are a few playlist title suggestions:
1. Dance Pop Hits
2. Alternative Fireworks
3. Retro Electro Vibes
4. Saturday Night Jam
5. Indie Dance Party
6. Variety Mix Tape
7. Electronic Rhythms
8. Poolside Anthems
9. 80s Era Love
10. Electric Pop Soundtrack


IndexError: list index out of range